## Feature Matching using ORB

* Oriented FAST and rotated BRIEF (ORB) is a fast robust local feature detector, first presented by Ethan Rublee et al. in 2011,[1] that can be used in computer vision tasks like object recognition or 3D reconstruction. It is based on the FAST keypoint detector and a modified version of the visual descriptor BRIEF (Binary Robust Independent Elementary Features). Its aim is to provide a fast and efficient alternative to SIFT.


In [ ]:
#Imports
import cv2 as cv2
import pandas as pd
import numpy as np
#import numpy as nm
from PIL import Image
import matplotlib.pyplot as plt
import os
%matplotlib inline

## Using orb on one image 
Display Keypoints 

In [ ]:
cols, rows = 4, 3
def grid_display(list_of_images, no_of_columns=2, figsize=(15,15), title = False):
    fig = plt.figure(figsize=figsize)
    column = 0
    z = 0
    for i in range(len(list_of_images)):
        column += 1
        #  check for end of column and create a new figure
        if column == no_of_columns+1:
            fig = plt.figure(figsize=figsize)
            column = 1
        fig.add_subplot(1, no_of_columns, column)
        if title:
            if i >= no_of_columns:
                plt.title(titles[z])
                z +=1
            else:
                plt.title(titles[i])
        plt.imshow(list_of_images[i])
        plt.axis('off')

In [ ]:
#Show ORB keypoints
image_all=[]
titles = ['original', 'ORB Detected', "Zoom Image"]
img = cv2.imread('../images/ID_0A3BSR6Q.jpg', 1)
image_all.append(img)
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints with ORB
kp = orb.detect(img,None)
# compute the descriptors with ORB
kp, des = orb.compute(img, kp)
# draw only keypoints location,not size and orientation
img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
image_all.append(img2)
img3 = img2[350:800,600:1250]
image_all.append(img3)
grid_display(image_all, 3, (35,35), title = True)

## Extract ORB features from all images

Currently disabled due to uncertainity of this method 

In [ ]:
#Feature extraction mit ORB
# Lets test it with 11 images first
'''
#read the images
import os
for dirname, _, filenames in os.walk('../test_images/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''

In [ ]:
#this takes 8 seconds for 11 images
#for all images it would take 31 minutes (2500 images)
'''
allFeatures=[]

for filename in os.listdir('../test_images/'):
    img = cv2.imread('../test_images/'+filename)
    img1 = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    mask = np.zeros(img1.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (5,5,235,235)
    cv2.grabCut(img1,mask,rect,bgdModel,fgdModel,10,cv2.GC_INIT_WITH_RECT) #image segmentation
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img1 = img1*mask2[:,:,np.newaxis]
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    #img1= cv2.GaussianBlur(img1,(5,5),cv2.BORDER_DEFAULT) 
    ll=cv2.equalizeHist(img1)
    orb = cv2.ORB_create(nfeatures=200,scoreType = cv2.ORB_HARRIS_SCORE)
    keypoints, descriptors = orb.detectAndCompute(ll, None) # here we compute the keypoints and descriptor
    allFeatures.append(descriptors)#array[ImageNb][FeatureNb]
'''   

---

## Feature Matching using ORB with color-masking 

In [ ]:
src_img = cv2.imread('../images/ID_0A3BSR6Q.jpg')

In [ ]:
# Function to display an image using matplotlib
def show_image(img, title, colorspace):
    dpi = 96
    figsize = (img.shape[1] / dpi, img.shape[0] / dpi)
    #figsize = (224,224)
    fig, ax = plt.subplots(figsize = figsize, dpi = dpi)
    if colorspace == 'RGB':
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), interpolation = 'spline16')
    if colorspace == 'gray':
        plt.imshow(img, cmap = 'gray')
    plt.title(title, fontsize = 12)
    ax.axis('off')
    plt.show() 

In [ ]:
show_image(src_img, 'Source image containing one turtle', 'RGB')

In [ ]:
# Change colorspace from BGR to HSV --> Hue , Saturation, Value (in HSV) --> object tracking based on color
src_img_hsv = cv2.cvtColor(src_img, cv2.COLOR_BGR2HSV)

# Define limits of yellow HSV values
yellow_lower = np.array([1, 30, 30])
yellow_upper = np.array([80, 55, 150])

# Filter the image and get the mask
mask = cv2.inRange(src_img_hsv, yellow_lower, yellow_upper)

show_image(mask, 'Yellow color filter mask', 'gray')

In [ ]:
# Remove white noise
kernel = np.ones((5, 5), np.uint8)
opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

show_image(opening, 'Morphological opening', 'gray')

In [ ]:
# Remove small black dots
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

show_image(closing, 'Morphological closing', 'gray')

In [ ]:
# Get back the fine boundary edges using dilation
kernel1 = np.ones((2, 2), np.uint8)
dilation = cv2.dilate(closing, kernel1, iterations = 1)

show_image(dilation, 'Morphological dilation', 'gray')

In [ ]:
contours, _ = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# There are 2 contours: outer one is the rectangle(ish) and inner one is the circle(ish)
# Get the outer contour (it has larger area than the inner contour)
c1 = max(contours, key = cv2.contourArea)

# Define the bounding rectangle around the contour
rect = cv2.minAreaRect(c1)

# Get the 4 corner coordinates of the rectangle
box = cv2.boxPoints(rect)
box = np.int0(box)

# Draw the bounding rectangle to show the marked object
temp_img = src_img.copy()
bdg_rect = cv2.drawContours(temp_img, [box], 0, (0, 0, 255), 2)

show_image(bdg_rect, 'Marked object to be extracted', 'RGB')

In [ ]:
# cv2.boxPoints(rect) returns the coordinates (x, y) as the following list:
# [[bottom right], [bottom left], [top left], [top right]]

width = box[0][0] - box[1][0]
height = box[1][1] - box[2][1]

src_pts = box.astype('float32')
dst_pts = np.array([[width, height],
                    [0, height],
                    [0, 0],
                    [width, 0]], dtype = 'float32')

# Get the transformation matrix
M = cv2.getPerspectiveTransform(src_pts, dst_pts)

# Apply the perspective transformation
warped = cv2.warpPerspective(src_img, M, (width, height))

# Save it as the query image
query_img = warped

In [ ]:
show_image(query_img, 'Query image', 'RGB')

In [ ]:
# Create an ORB object
orb = cv2.ORB_create()

# Detect and visualize the features
features = orb.detect(query_img, None)
f_img = cv2.drawKeypoints(query_img, features, None, color = (0, 255, 0), flags = 0)

show_image(f_img, 'Detected features', 'RGB')

In [ ]:
# Function to match features and find the object
def match_feature_find_object(query_img, train_img, min_matches): 
    # Create an ORB object
    orb = cv2.ORB_create(nfeatures=100000)
    
    features1, des1 = orb.detectAndCompute(query_img, None)
    features2, des2 = orb.detectAndCompute(train_img, None)

    # Create Brute-Force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(des1, des2, k = 2)
    
    # Nearest neighbour ratio test to find good matches
    good = []    
    good_without_lists = []    
    matches = [match for match in matches if len(match) == 2] 
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
            good_without_lists.append(m)
         
    if len(good) >= min_matches:
        # Draw a polygon around the recognized object
        src_pts = np.float32([features1[m.queryIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        dst_pts = np.float32([features2[m.trainIdx].pt for m in good_without_lists]).reshape(-1, 1, 2)
        
        # Get the transformation matrix
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
               
        # Find the perspective transformation to get the corresponding points
        h, w = query_img.shape[:2]
        pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)
        
        train_img = cv2.polylines(train_img, [np.int32(dst)], True, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        print('Not enough good matches are found - {}/{}'.format(len(good), min_matches))
            
    result_img = cv2.drawMatchesKnn(query_img, features1, train_img, features2, good, None, flags = 2)
    
    show_image(result_img, 'Feature matching and object recognition', 'RGB')

In [ ]:
train_img = cv2.imread('../images/ID_0A3BSR6Q.jpg')
match_feature_find_object(query_img, train_img, 5)



## ORB Circles

In [ ]:
dataset_path = '../images/'
img_first = cv2.imread(os.path.join(dataset_path, 'ID_0A3BSR6Q.jpg'))
img_first = cv2.cvtColor(img_first, cv2.COLOR_BGR2RGB)  # Convert from cv's BRG default color order to RGB

orb = cv2.ORB_create()  # OpenCV 3 backward incompatibility: Do not create a detector with `cv2.ORB()`.
key_points, description = orb.detectAndCompute(img_first, None)
img_first_keypoints = cv2.drawKeypoints(img_first, 
                                           key_points, 
                                           img_first, 
                                           flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) # Draw circles.
plt.figure(figsize=(16, 16))
plt.title('ORB Interest Points')
plt.imshow(img_first_keypoints); plt.show()

In [ ]:
def image_detect_and_compute(detector, img1, img2):
    """Detect and compute interest points and their descriptors."""
    img1 = cv2.imread(os.path.join(dataset_path, img1))
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    kp1, des1 = detector.detectAndCompute(img1, None)
    
    img2 = cv2.imread(os.path.join(dataset_path, img2))
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    kp2, des2 = detector.detectAndCompute(img2, None)
    
    
    
    return img1, kp1, des1,img2, kp2, des2
    

def draw_image_matches(detector, img1, img2, nmatches=50):
    """Draw ORB feature matches of the given two images."""
    img1, kp1, des1, img2, kp2, des2 = image_detect_and_compute(detector,img1, img2)
    #img2, kp2, des2 = image_detect_and_compute(detector,img2)
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key = lambda x: x.distance) # Sort matches by distance.  Best come first.
    print(matches[:5])
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:nmatches], img2, flags=2) # Show top 10 matches
    plt.figure(figsize=(16, 16))
    plt.title(type(detector))
    plt.imshow(img_matches); plt.show()
orb = cv2.ORB_create()
    
#'ID_0A3BSR6Q.jpg'
#'ID_0AEH3RAW.jpg'
draw_image_matches(orb, 'ID_0AEH3RAW.jpg', 'ID_0A3BSR6Q.jpg')